In [1]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import pandas as pd
import numpy as np

In [2]:
### Load the trained model, scaler pickle,onehot
model=load_model('r_model.h5')

## load the encoder and scaler
with open('r_onehot_encoder_geo.pkl','rb') as file:
    label_encoder_geo=pickle.load(file)

with open('r_label_encoder_gender.pkl', 'rb') as file:
    label_encoder_gender = pickle.load(file)

with open('r_scaler.pkl', 'rb') as file:
    scaler = pickle.load(file)

In [3]:
# Example input data
input_data = {
    'CreditScore': 501,
    'Geography': 'Spain',
    'Gender': 'Male',
    'Age': 44,
    'Tenure': 8,
    'Balance': 113755.78,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 0,
    'Exited': 1
}

In [4]:
# One-hot encode 'Geography'
geo_encoded = label_encoder_geo.transform([[input_data['Geography']]]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=label_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df


i:\project\tensorflow ann\Churn_Classification_Using_Tensorflow_ANN\.envv\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,0.0,0.0,1.0


In [5]:
input_df=pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,Exited
0,501,Spain,Male,44,8,113755.78,2,1,0,1


In [6]:
## Encode categorical variables
input_df['Gender']=label_encoder_gender.transform(input_df['Gender'])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,Exited
0,501,Spain,1,44,8,113755.78,2,1,0,1


In [7]:
## concatination one hot encoded 
input_df=pd.concat([input_df.drop("Geography",axis=1),geo_encoded_df],axis=1)
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,Exited,Geography_France,Geography_Germany,Geography_Spain
0,501,1,44,8,113755.78,2,1,0,1,0.0,0.0,1.0


In [8]:
## Scaling the input data
input_scaled=scaler.transform(input_df)
input_scaled

array([[-1.56338053,  0.91324755,  0.48508334,  1.03972843,  0.60287204,
         0.80843615,  0.64920267, -1.02583358,  1.96626042, -0.99850112,
        -0.57946723,  1.73494238]])

In [9]:
## PRedict churn
regression=model.predict(input_scaled)
regression

1/1 [==============================] - 5s 5s/step


array([[107031.6]], dtype=float32)

In [10]:
predict_salary = regression[0][0]

In [11]:
predict_salary = int(predict_salary)

In [ ]:
print(f"Predicted Salary for this customer is {predict_salary:.2f}")

Predicted Salary for this customer is 107031
